In [56]:
import pandas as pd
import pandas_market_calendars as mcal
import datetime
from Tools.Trade_Functions import *
import numpy as np
import plotly.express as px


In [2]:
longFlag =1
shortFlag =1

In [43]:
def pick_trade(TradeData,RunDate,NumStocks,stopLoss=-0.01,holdTime=0,TimeInput=5,debug=False,numDaysToCheckStopLoss=5):
    #Calculate pct change
    global longFlag
    global shortFlag
    
    
    
    PctChange = get_SortedPctChange(RunDate,TimeInput,debug=False)
    
    #select first N stocks and caculate percent retruns
    ShortStockList = PctChange["Ticker"].iloc[:NumStocks]
    LongStockList = PctChange["Ticker"].iloc[len(PctChange["Ticker"])-NumStocks:]
    
    longSum=0
    shortSum=0
    LongTradeInfo = None
    ShortTradeInfo = None
    
    if longFlag:
        LongTradeInfo = getLongReturns(RunDate,LongStockList,stopLoss=stopLoss,holdTime=holdTime,debug=debug,numDaysToCheckStopLoss=5)
        longSum = sum(LongTradeInfo["Returns"])

    if shortFlag:
        ShortTradeInfo = getShortReturns(RunDate,ShortStockList,stopLoss=stopLoss,holdTime=holdTime,debug=debug,numDaysToCheckStopLoss=5)
        shortSum = sum(ShortTradeInfo["Returns"])
    
    if longSum<0 and shortSum>= 0:
        longFlag = 1
        shortFlag = 0
        
    if shortSum<0 and longSum>=0:
        shortFlag =1
        longFlag = 0

    
    TotalReturn = (longSum + shortSum) / NumStocks 
    
    return TotalReturn, pd.concat([TradeData,LongTradeInfo,ShortTradeInfo])

In [44]:
framedCal= get_calendar()

In [47]:
TotalReturnList = list()
DateReturnList = list()
TimeInput = 6
NumStocks= 20
holdTime=4
stopLoss= -1
#must be less than holdtime
numDaysToCheckStopLoss= holdTime
TradeData=pd.DataFrame()
cal=list()
for i in range(252,len(framedCal),holdTime+1):
    cal.append(framedCal[i])

#Anual Range
#year=2022
#yearlyDates = [date for date in cal if date.startswith(str(year))]
#DateRange = [yearlyDates[0], yearlyDates[-1]]

#Set custome Range
DateRange=[cal[1],cal[-1]]
#DateRange = [cal[51], cal[-1]]


DateRange=[cal.index(DateRange[0]),cal.index(DateRange[1])]
for day in cal[DateRange[0]:DateRange[1]]:
    dailyReturn,TradeData = pick_trade(TradeData,RunDate=day,NumStocks=NumStocks,stopLoss=stopLoss,TimeInput=TimeInput,holdTime=holdTime,debug=False,numDaysToCheckStopLoss=numDaysToCheckStopLoss)
    TotalReturnList.append(dailyReturn)
    
TradeData=TradeData.reset_index(drop=True)
returnDf = pd.DataFrame({'Date': pd.to_datetime(cal[DateRange[0]:DateRange[1]]),'Return':TotalReturnList})


M = 1
returnVal=[]
for i in returnDf["Return"]:
    M = M*(1+i)
    returnVal.append(M)
returnDf=pd.concat([returnDf,pd.DataFrame({"ReturnVals":returnVal})],axis=1)





In [48]:

fig = px.line(returnDf, x='Date', y='ReturnVals')

fig.update_layout(title='Returns Over Time',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()

In [ ]:
i = f"{lookback}lookback"

In [49]:
for year in range(2015,2025):
    returnDf_year = returnDf[returnDf['Date'].dt.year == year]
    returnDf_year.loc[:, 'Return'] = returnDf_year['Return'].astype(float)
    print(year,np.prod(1 + returnDf_year['Return']))

2015 1.0706291718534613
2016 1.134392826914917
2017 1.2515418508160403
2018 0.9541074427345735
2019 1.3691375316459287
2020 1.692351828230505
2021 1.3906677662903952
2022 0.8264567972925585
2023 1.3401408675473843
2024 1.1939417288116614


In [112]:
print("SharpeRatio",( np.mean(returnDf["Return"]) * 252 - 0.05) / (np.std(returnDf["Return"]) * 252**0.5))

SharpeRatio -0.15898476649452933


In [35]:
TradeData.to_csv("TradeData.csv")

In [114]:
start = getPrice("2015-01-12","AMZN","Open")
end = getPrice("2015-01-16","AMZN","Close")
print(f"Start: {start}, End: {end}, Return: {(end-start)/start}")

Start: 14.878, End: 14.537, Return: -0.02291974727785988


In [115]:
(end-start)/start -0.0004

-0.023319747277859883

In [52]:
max_drawdown = calculate_mdd(returnDf["Return"])
max_drawdown

-0.34495366057607757